# Pyridine Parameter Estimation (Direct Multiple Shooting + GN)

This notebook sets up the Pyridine dynamic system, generates synthetic noisy data, formulates the direct multiple shooting problem, and solves it using the Generalized Gauss-Newton method. We then compare the estimated trajectory with the true data.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from multiple_shooting import setup_multiple_shooting
from cnlls_solver import solve_cnlls_gauss_newton
from pyridine import make_pyridine_integrator, get_initial_state, get_true_parameters, simulate_pyridine
from utils import add_noise, evaluate_fit

## Define Pyridine System and Generate Data

We define the time horizon and generate synthetic noisy measurements from the true system.

In [ ]:
# Problem setup
N = 20          # number of shooting intervals
T = 6.0         # final time
t_grid = np.linspace(0, T, N+1)  # time grid of length N+1
dt = t_grid[1] - t_grid[0]       # time step

# True initial state and parameters
x0_true = get_initial_state()
p_true = get_true_parameters()

# Create integrator for the system
F = make_pyridine_integrator(dt)

# Simulate true trajectory (no noise)
y_true = simulate_pyridine(F, x0_true, p_true, N)

# Add Gaussian noise to simulated measurements (keeping initial exact)
noise_level = 0.01
y_meas_noisy = y_true.copy()
y_meas_noisy[1:] = add_noise(y_true[1:], noise_level=noise_level)

## Set Up Multiple Shooting Problem

We create symbolic variables for the multiple shooting formulation.

In [ ]:
nx = x0_true.size
np_p = p_true.size

# Setup multiple shooting
w_sym, X_end, g_sym, F2, F3, S_vars, P_var = setup_multiple_shooting(F, N, nx, np_p)

# Construct the residual vector F1 (difference between model and measurements)
# Note: y_meas_noisy has shape (N+1, 7), we compare interval end states to measurements at next time
F1 = []
for i in range(N):
    Yi = X_end[i] - y_meas_noisy[i+1]  # CASADI will treat the numpy array as constant
    F1.append(Yi)
F1 = ca.vertcat(*F1)

## Solve with Generalized Gauss-Newton

We use the GGN solver to estimate the initial states and parameters.

In [ ]:
# Initial guess: use noisy measurements (except last) for S_vars and ones for parameters
w0 = np.concatenate([y_meas_noisy[:-1].reshape(-1), np.ones(np_p)])

# Solve using Gauss-Newton method
result_gn = solve_cnlls_gauss_newton(w=w_sym, F1=F1, g=g_sym, F2=F2, F3=F3, w0=w0, max_iter=20, tol=1e-6)
w_opt = result_gn['x']
p_opt = w_opt[-np_p:]  # estimated parameters

print("True parameters:     ", p_true)
print("Estimated parameters:", np.round(p_opt, 4))
print("Converged:", result_gn['converged'], " in iterations:", result_gn['iterations'])

## Compare Trajectories

Simulate the system with the estimated parameters and compare the trajectories.

In [ ]:
# Simulate with estimated parameters
y_est = simulate_pyridine(F, x0_true, p_opt, N)

# Evaluate fit (RMSE, MAE) for each species
evaluate_fit(y_true, y_est, t_grid, label="GGN")

# Plot comparison for selected species
plt.figure(figsize=(8,6))
species_indices = [0, 1, 3, 5, 6]  # indices of species to plot
markers = ['o', 'D', 's', '^', 'v']
labels = ['Pyridine', 'Piperidin', 'N-Pentylpiperidin', 'Ammonia', 'Pentan']

for idx, marker, label in zip(species_indices, markers, labels):
    # Measured (noisy) data
    plt.plot(t_grid, y_meas_noisy[:, idx], marker+':', label=f"{label} (meas)", alpha=0.6)
    # Estimated trajectory
    plt.plot(t_grid, y_est[:, idx], '-', label=f"{label} (est)")

plt.xlabel('Time')
plt.ylabel('Concentration')
plt.title('Pyridine System: Measured vs Estimated Trajectories')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

The above plot shows the noisy measurements (markers) and the fitted trajectories (lines) for selected species. The printed errors (RMSE and MAE) quantify the fit quality for each species.